In [87]:
# !pip install selenium

In [88]:
# !pip install webdriver_manager

In [89]:
# !pip install pandas

In [90]:
# !pip install seaborn

In [91]:
# !pip install sklearn

In [92]:
# !pip install scikit-learn

In [93]:
# !pip install transformers

In [94]:
# conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

In [95]:
# conda update -n base -c defaults conda

In [96]:
# !pip install chardet

In [97]:
# !pip install huggingface_hub

In [98]:
# !pip install finbert

In [99]:
# !pip install tensorflow

In [100]:
import warnings
import random
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,  AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
# 허깅페이스 - finbert
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pands as pd

In [ ]:
warnings.filterwarnings('ignore') # warning 무시

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
def set_chrome_driver(headless=True):
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


In [ ]:
def crawl_page(url):
    try:
        driver = set_chrome_driver(False) # Chrome 웹 드라이버 인스턴스 생성
        driver.get(url)  # 지정된 URL을 웹 드라이버에서 열기
        # 요소 변경 가능
        article_page = driver.find_element(By.CLASS_NAME, 'articlePage') # class 이름으로 뉴스 기사 페이지 요소 찾기
        text = article_page.text # 뉴스 기사 페이지 요소에서 텍스트 추출
        driver.close() # 텍스트 추출 후 웹 드라이버 닫기
    except NoSuchElementException: 
        text = ""  # 뉴스 기사 페이지 요소가 없을 경우 텍스트를 빈 문자열로 설정
    except TimeoutException:
        text = "" # 웹페이지가 제대로 로드되지 않으면 텍스트를 빈 문자열로 설정
    return text  # 추출된 텍스트 반환

In [ ]:
page = crawl_page('https://www.investing.com/analysis/traders-send-wheat-prices-spiking-as-allied-tanks-aid-to-roll-into-ukraine-200634894')  # 뉴스 페이지 크롤링
text_data = str(page) # 크롤링한 데이터를 문자열로 변환

# 텍스트 데이터 정리 및 전처리
text_data = re.sub('<.*?>', '', text_data) # HTML 태그 제거
text_data = re.sub(r'[^\w\s]', '', text_data) # 특수문자 제거
# 필요에 따라 다른 전처리 단계를 수행

In [ ]:
encoded_text = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# 감정 분석을 위해 인코딩된 텍스트를 모델에 전달  
# 출력 = 모델

outputs = model(**encoded_text)
logits = outputs.logits
predicted_class_idx = logits.argmax(dim=1).item()

In [ ]:
# 모델의 출력에서 예측된 감정 레이블 가져오기

sentiment_labels = ["Negative", "Neutral", "Positive"]
predicted_sentiment = sentiment_labels[predicted_class_idx]

print("Predicted Sentiment: {}".format(predicted_sentiment))

In [ ]:
def show_random_headlines(total_number, df):
    
    # Get the random number of reviews
    n_reviews = df.sample(total_number)
    
    # Print each one of the reviews
    for val in list(n_reviews.index):
        print("Reviews #°{}".format(val))
        print(" - Sentiment: {}".format(df.iloc[val]["sentiment"]))
        print(" - News Headline: {}".format(df.iloc[val]["NewsHeadline"]))
        print("")

In [ ]:
show_random_headlines(total_number, df)

In [ ]:
# 크롤링된 데이터 및 감성 라벨로 df DataFrame 생성 및 채우기
df = pd.DataFrame({'NewsHeadline': news_headlines, 'sentiment': sentiment_labels})